In [1]:
import os
import json
import random
import numpy as np
import pandas as pd
import math
import torch
import gpytorch
import sys
import re
import statistics
from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [2]:
import simple_gp
from Kidera import kidera

In [3]:
# df = pd.read_csv('data/DDG_Dataset.csv')
df = pd.read_csv('data/DDG_Dataset_mini.csv')
df = df[df['Temperature'] != "'-"].reset_index(drop=True)
df['Temperature'] = df['Temperature'].astype('float64')
ndata = df.shape[0]
print(df.columns)
print(ndata)
df

Index(['Rowid', 'ID', 'Protein Name', 'Mutation', 'Chain', 'PDB',
       'Temperature', 'pH', 'ΔΔG', 'Reference', 'A', 'R', 'N', 'D', 'C', 'Q',
       'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V',
       'ASP', 'PHE', 'GLN', 'LYS', 'ILE', 'TYR', 'GLY', 'ASN', 'ARG', 'LEU',
       'TRP', 'ALA', 'THR', 'VAL', 'HIS', 'CYS', 'GLU', 'MET', 'PRO', 'SER'],
      dtype='object')
854


,Rowid,ID,Protein Name,Mutation,Chain,PDB,Temperature,pH,ΔΔG,Reference,...,TRP,ALA,THR,VAL,HIS,CYS,GLU,MET,PRO,SER
0,1,1,Tryptophan synthase alpha chain,E49M,A,1WQ5,298.95,7.0,4.60,PMID: 378988,...,0.0,4.0,2.0,4.0,0.0,0.0,0.0,1.0,2.0,2.0
1,11,13,Cellular tumor antigen p53,M237I,A,1TUP,283.15,7.2,-3.18,PMID: 1203434,...,0.0,2.0,1.0,3.0,3.0,6.0,2.0,2.0,2.0,3.0
2,21,31,Ribonuclease HI,S68T,A,2RN2,298.15,5.5,0.10,PMID: 1317795,...,2.0,4.0,2.0,4.0,1.0,1.0,5.0,0.0,1.0,1.0
3,31,45,Ribonuclease HI,H62P,A,2RN2,298.15,5.5,0.74,PMID: 1328237,...,0.0,2.0,0.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0
4,41,59,Ribonuclease HI,K95G,A,2RN2,322.95,3.0,1.70,PMID: 1331044,...,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,8531,13279,Transcriptional repressor arc,K6A,A,1ARR,298.15,7.5,0.40,PMID: 7664079,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0
850,8541,13289,Transcriptional repressor arc,E17A,A,1ARR,298.15,7.5,-0.50,PMID: 7664079,...,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0
851,8551,13299,Transcriptional repressor arc,R23A,A,1ARR,298.15,7.5,-1.10,PMID: 7664079,...,1.0,1.0,0.0,4.0,0.0,0.0,4.0,0.0,1.0,1.0
852,8561,13309,Transcriptional repressor arc,N29A,A,1ARR,298.15,7.5,-1.60,PMID: 7664079,...,0.0,1.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,2.0


In [4]:
kidera.symbol_lookup
kidera_table = pd.read_csv('Kidera/kidera.csv', header=None).set_index(0)
kidera_table

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
ALA,-1.56,-1.67,-0.97,-0.27,-0.93,-0.78,-0.20,-0.08,0.21,-0.48
ARG,0.22,1.27,1.37,1.87,-1.70,0.46,0.92,-0.39,0.23,0.93
ASN,1.14,-0.07,-0.12,0.81,0.18,0.37,-0.09,1.23,1.10,-1.73
ASP,0.58,-0.22,-1.58,0.81,-0.92,0.15,-1.52,0.47,0.76,0.70
CYS,0.12,-0.89,0.45,-1.05,-0.71,2.41,1.52,-0.69,1.13,1.10
GLN,-0.47,0.24,0.07,1.10,1.10,0.59,0.84,-0.71,-0.03,-2.33
GLU,-1.45,0.19,-1.61,1.17,-1.31,0.40,0.04,0.38,-0.35,-0.12
GLY,1.46,-1.96,-0.23,-0.16,0.10,-0.11,1.32,2.36,-1.66,0.46
HIS,-0.41,0.52,-0.28,0.28,1.61,1.01,-1.85,0.47,1.13,1.63


In [5]:
%%time 
around_acid = np.zeros((ndata, 10))
for index, row in df.iterrows():
    # print(index)
    # print(row)
    for acid, r in kidera_table.iterrows():
        # print(acid)
        # print(r)
        # print(row[acid])
        # print(row[acid] * r)
        around_acid[index, :] += row[acid] * r
        # break
    # break
    
around_acid

CPU times: user 4.11 s, sys: 31.4 ms, total: 4.14 s
Wall time: 4.18 s


array([[-11.27, -15.66,  15.32, ...,   3.88,   6.42,  -5.23],
       [ -0.37,  -3.03,   7.07, ...,  -8.51,   9.49,  10.99],
       [-12.16,  -8.73,   0.12, ...,   3.77,  -3.07,  -2.43],
       ...,
       [ -5.54,  -2.18,   0.81, ...,   1.25,  -2.72,   0.77],
       [ -6.97,  -2.64,   5.91, ...,  -0.91,  -4.53,   1.32],
       [ -6.23,   2.6 ,   3.67, ...,  -3.63,  -0.9 ,   4.52]])

In [6]:
data_x = df.iloc[:, :30].drop(columns=['Rowid', 'ID', 'Protein Name', 'Mutation', 'Chain', 'PDB', 'ΔΔG', 'Reference']).to_numpy()
data_x = np.concatenate([data_x, around_acid], 1)
data_y = df['ΔΔG'].to_numpy()
data_x

array([[298.95,   7.  ,   0.  , ...,   3.88,   6.42,  -5.23],
       [283.15,   7.2 ,   0.  , ...,  -8.51,   9.49,  10.99],
       [298.15,   5.5 ,   0.  , ...,   3.77,  -3.07,  -2.43],
       ...,
       [298.15,   7.5 ,   1.  , ...,   1.25,  -2.72,   0.77],
       [298.15,   7.5 ,   1.  , ...,  -0.91,  -4.53,   1.32],
       [298.15,   7.5 ,   1.  , ...,  -3.63,  -0.9 ,   4.52]])

In [7]:
# print(np.unique(data_x[:, 0]))
# print(len(np.unique(data_x[:, 0])))
# random.choices(sorted(np.unique(data_x[:, 0])), k=200)
# random.choices(range(2), k=200)

In [8]:
os.system('mkdir -p data/boruta')
for r in range(data_x.shape[1]):
    tmp_data_x = np.copy(data_x)
    random_list = random.choices(sorted(np.unique(tmp_data_x[:, r])), k=ndata)
    tmp_data_x[:, r] = random_list
    pred_y, lower_y, upper_y = simple_gp.solve(tmp_data_x, data_y, 5, 5)
    # pred_y, lower_y, upper_y = simple_gp.solve(data_x, data_y, 5, 100)
    tmp_df = df.copy()
    tmp_df['rand_items'] = random_list
    tmp_df['pred_ΔΔG'] = pred_y
    tmp_df['lower_ΔΔG'] = lower_y
    tmp_df['upper_ΔΔG'] = upper_y
    tmp_df.to_csv('data/boruta/pred_DDG_Dataset_gpytorch_Kidera_{}.csv'.format(r))